## Sklearn MME W/ Boto3 SDK

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib

#Load data
boston = datasets.load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df['MEDV'] = boston.target 

#Split Model
X = df.drop(['MEDV'], axis = 1) 
y = df['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [2]:
#Model Creation
lm = LinearRegression()
lm.fit(X_train,y_train)


with open('model.joblib', 'wb') as f:
    joblib.dump(lm,f)


with open('model.joblib', 'rb') as f:
    predictor = joblib.load(f)

print("Testing following input: ")
print(X_test[0:1])
sampInput = [[0.09178, 0.0, 4.05, 0.0, 0.51, 6.416, 84.1, 2.6463, 5.0, 296.0, 16.6, 395.5, 9.04]]
print(type(sampInput))
print(predictor.predict(sampInput))

Testing following input: 
        CRIM   ZN  INDUS  CHAS   NOX     RM   AGE     DIS  RAD    TAX  \
173  0.09178  0.0   4.05   0.0  0.51  6.416  84.1  2.6463  5.0  296.0   

     PTRATIO      B  LSTAT  
173     16.6  395.5   9.04  
<class 'list'>
[28.99672362]


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [3]:
%%writefile inference.py
import joblib
import os
import json

"""
Deserialize fitted model
"""
def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

"""
input_fn
    request_body: The body of the request sent to the model.
    request_content_type: (string) specifies the format/variable type of the request
"""
def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        request_body = json.loads(request_body)
        inpVar = request_body['Input']
        return inpVar
    else:
        raise ValueError("This model only supports application/json input")

"""
predict_fn
    input_data: returned array from input_fn above
    model (sklearn model) returned model loaded from model_fn above
"""
def predict_fn(input_data, model):
    return model.predict(input_data)

"""
output_fn
    prediction: the returned value from predict_fn above
    content_type: the content type the endpoint expects to be returned. Ex: JSON, string
"""

def output_fn(prediction, content_type):
    res = int(prediction[0])
    respJSON = {'Output': res}
    return respJSON

Writing inference.py


In [4]:
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess


#Setup
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = boto_session.region_name
print(region)
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

us-east-1


In [5]:
#Build tar file with model data + inference code
bashCommand = "tar -cvpzf model.tar.gz model.joblib"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [6]:
#Build tar file with model data + inference code
bashCommand = "tar -cvpzf source.tar.gz inference.py"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [7]:
#Bucket for model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

sagemaker-us-east-1-474422712127


In [8]:
%%sh

s3_bucket='sagemaker-us-east-1-474422712127'

for i in {0..5}
do
  aws s3 cp model.tar.gz s3://$s3_bucket/mme-sklearn-four-latest/sklearn-$i.tar.gz 
done

upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/sklearn-0.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/sklearn-1.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/sklearn-2.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/sklearn-3.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/sklearn-4.tar.gz
upload: ./model.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/sklearn-5.tar.gz


In [9]:
!aws s3 ls s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/

2022-08-04 14:45:05       1059 sklearn-0.tar.gz
2022-08-04 14:45:05       1059 sklearn-1.tar.gz
2022-08-04 14:45:06       1059 sklearn-2.tar.gz
2022-08-04 14:45:06       1059 sklearn-3.tar.gz
2022-08-04 14:45:06       1059 sklearn-4.tar.gz
2022-08-04 14:45:07       1059 sklearn-5.tar.gz


In [10]:
!aws s3 cp source.tar.gz s3://sagemaker-us-east-1-474422712127/mme-sklearn-inf/source.tar.gz

upload: ./source.tar.gz to s3://sagemaker-us-east-1-474422712127/mme-sklearn-inf/source.tar.gz


In [27]:
!aws s3 ls s3://sagemaker-us-east-1-474422712127/mme-sklearn-inf/

2022-07-11 17:57:04        619 source.tar.gz


In [11]:
source_dir = 's3://sagemaker-us-east-1-474422712127/mme-sklearn-inf/source.tar.gz'

In [12]:
s3_bucket='sagemaker-us-east-1-474422712127'
model_url = 's3://{}/mme-sklearn-four-latest/'.format(s3_bucket) ## MODEL S3 URL

In [13]:
!aws s3 ls {model_url}

2022-08-04 14:45:05       1059 sklearn-0.tar.gz
2022-08-04 14:45:05       1059 sklearn-1.tar.gz
2022-08-04 14:45:06       1059 sklearn-2.tar.gz
2022-08-04 14:45:06       1059 sklearn-3.tar.gz
2022-08-04 14:45:06       1059 sklearn-4.tar.gz
2022-08-04 14:45:07       1059 sklearn-5.tar.gz


In [14]:
# retrieve sklearn image
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="0.23-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)
image_uri

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

In [15]:
from time import gmtime, strftime
model_name = 'mme-source' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "MultiModel",
            "ModelDataUrl": model_url,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': source_dir,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

Model name: mme-source2022-08-04-14-45-32
Model data Url: s3://sagemaker-us-east-1-474422712127/mme-sklearn-four-latest/
Model Arn: arn:aws:sagemaker:us-east-1:474422712127:model/mme-source2022-08-04-14-45-32


In [16]:
#Step 2: EPC Creation
sklearn_epc_name = "mme-source" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.large",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint-config/mme-source2022-08-04-14-45-35


In [17]:
#Step 3: EP Creation
endpoint_name = "mme-source" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint/mme-source2022-08-04-14-45-38


In [18]:
#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'mme-source2022-08-04-14-45-38', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:474422712127:endpoint/mme-source2022-08-04-14-45-38', 'EndpointConfigName': 'mme-source2022-08-04-14-45-35', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:15d406612ed5c542743af479db5c98392cdf30ee46080c8f9e4857606c6697ff', 'ResolutionTime': datetime.datetime(2022, 8, 4, 14, 45, 41, 370000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', '

In [30]:
import boto3
import json

runtime_client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {"Input": [[0.09178, 0.0, 4.05, 0.0, 0.51, 6.416, 84.1, 2.6463, 5.0, 296.0, 16.6, 395.5, 9.04]]}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
endpoint_name = "mme-source2022-08-04-14-45-38"

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    TargetModel = "sklearn-2.tar.gz",
    Body=payload)
result = json.loads(response['Body'].read().decode())['Output']
print(result)

28
